# YOLO Training Interface in Google Colab
This notebook demonstrates how to set up and run the YOLO Training Interface in Google Colab.

## Step 1: Install Dependencies

In [ ]:
# Install the required dependencies
!python -m pip install ultralytics>=8.0.0 rich>=10.0.0 pyyaml>=6.0 pandas>=1.3.0 matplotlib>=3.4.0 numpy>=1.20.0 ray>=2.0.0 gdown>=4.6.0

# Import necessary packages for this notebook
import os
import yaml

## Step 2: Upload the required files
Upload the following files to this Colab session:
1. program.py - The main YOLO training interface
2. setup_env.py - The environment setup script

In [ ]:
# Download files directly from Github repository
!wget https://raw.githubusercontent.com/d-quint/colab/refs/heads/main/program.py -O program.py
!wget https://raw.githubusercontent.com/d-quint/colab/refs/heads/main/setup_env.py -O setup_env.py

# Verify the files were downloaded correctly
!ls -la program.py setup_env.py

## Step 3: Set up the Environment

In [ ]:
# Run the setup script to create the necessary directory structure
%run setup_env.py --download_models

## Fix Sample Model File

The setup script creates a placeholder .pt file that isn't a real model. Let's replace it with a real YOLOv8 model.

In [ ]:
# Fix the sample weight file by replacing it with a real YOLOv8 model
import os
from ultralytics import YOLO

# Directory for the sample model
sample_weights_dir = "/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights"
sample_weights_file = os.path.join(sample_weights_dir, "last.pt")

# Check if the sample weight file exists and is just a placeholder
if os.path.exists(sample_weights_file):
    with open(sample_weights_file, 'r') as f:
        content = f.read()
        if content.startswith("#"):
            print("Detected placeholder model file. Replacing with a real YOLOv8n model...")
            
            # Download a real YOLOv8n model
            model = YOLO("yolov8n.pt")
            
            # Save the real model to the sample weights directory
            os.remove(sample_weights_file)  # Remove the placeholder
            model.save(sample_weights_file)  # Save the real model
            
            print(f"✅ Successfully replaced placeholder with real model at: {sample_weights_file}")
else:
    print(f"Sample weights directory not found. Creating it...")
    os.makedirs(sample_weights_dir, exist_ok=True)
    
    # Download and save a real YOLOv8n model
    model = YOLO("yolov8n.pt")
    model.save(sample_weights_file)
    
    print(f"✅ Created real model at: {sample_weights_file}")

## Step 4: Prepare Your Dataset
You can either use the sample dataset structure created by the setup script, or upload your own dataset. The dataset should follow the YOLOv8 format:

```
datasets/
  your_dataset/
    images/
      train/
      val/
      test/ (optional)
    labels/
      train/
      val/
      test/ (optional)
```

And update the dataset.yaml file to point to your dataset.

In [ ]:
# Option 1: Download dataset from Google Drive using gdown
import gdown

# Google Drive file ID
file_id = "19Mzi4Ueqyh3i634c9oUvP4NEskwoB86z"
output_path = "dataset.zip"

# Download the file
print("Downloading dataset from Google Drive...")
gdown.download(id=file_id, output=output_path, quiet=False)

# Extract to the datasets directory
!mkdir -p /content/drive/MyDrive/YOLO_Training/datasets/custom_dataset
!unzip -q dataset.zip -d /content/drive/MyDrive/YOLO_Training/datasets/custom_dataset
!rm dataset.zip

# Check the downloaded dataset structure
print("\nDownloaded dataset structure:")
!ls -la /content/drive/MyDrive/YOLO_Training/datasets/custom_dataset

# Option 2: Download a dataset directly from Roboflow (alternative method)
# !curl -L "https://app.roboflow.com/ds/9oNEAZK93B?key=BjWc8uu2z1" > roboflow.zip
# !unzip -q roboflow.zip -d /content/drive/MyDrive/YOLO_Training/datasets/roboflow
# !rm roboflow.zip
# !ls -la /content/drive/MyDrive/YOLO_Training/datasets/roboflow

# Option 3: Upload a dataset from your local machine
# from google.colab import files
# uploaded = files.upload()  # Choose your dataset zip file
# !unzip -q dataset.zip -d /content/drive/MyDrive/YOLO_Training/datasets/

In [ ]:
# Update the dataset.yaml file to point to the downloaded dataset
import yaml
import os

# Path where we want to save our dataset.yaml
dataset_yaml = "/content/drive/MyDrive/YOLO_Training/dataset.yaml"

# Look for yaml files in the dataset directory
custom_dataset_dir = "/content/drive/MyDrive/YOLO_Training/datasets/custom_dataset"
yaml_files = [f for f in os.listdir(custom_dataset_dir) if f.endswith('.yaml')]

if yaml_files:
    # Use the first yaml file found
    dataset_config_path = os.path.join(custom_dataset_dir, yaml_files[0])
    print(f"Found dataset config: {dataset_config_path}")
    
    # Read the yaml file
    with open(dataset_config_path, 'r') as f:
        data = yaml.safe_load(f)
    
    # Update paths to be absolute
    data['path'] = custom_dataset_dir
    
    # Save to our main dataset.yaml location
    with open(dataset_yaml, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
    
    print(f"Updated {dataset_yaml} with dataset configuration")
    print("\nDataset configuration:")
    with open(dataset_yaml, 'r') as f:
        print(f.read())
else:
    print(f"Warning: No YAML configuration found in {custom_dataset_dir}")
    print("Creating a default dataset.yaml file. You may need to update it manually.")
    
    # Create a default dataset.yaml
    default_config = {
        'path': custom_dataset_dir,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {
            0: 'object',  # Default class name
        }
    }
    
    with open(dataset_yaml, 'w') as f:
        yaml.dump(default_config, f, default_flow_style=False)
    
    print("\nDefault dataset configuration created:")
    with open(dataset_yaml, 'r') as f:
        print(f.read())

## Step 5: Run the YOLO Training Interface

In [ ]:
# Change to the directory where the program.py is located
%cd /content/drive/MyDrive/YOLO_Training

# Run the program
%run program.py

## Optional: Monitor Training with TensorBoard
YOLOv8 automatically creates TensorBoard logs which you can view in Colab.

In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
%tensorboard --logdir /content/drive/MyDrive/YOLO_Training/runs

## Troubleshooting

### Error: "invalid load key, '#'"

If you encounter the error message "invalid load key, '#'" when trying to run hyperparameter tuning, this typically happens because the setup script creates placeholder model files instead of real ones. The fix is included in this notebook (see "Fix Sample Model File" section above).

If you still encounter this issue:

1. Check if the model file is a real PyTorch model and not a placeholder:

In [ ]:
# Check if the model file is valid
def check_model_file(model_path):
    try:
        # Try opening as text
        with open(model_path, 'r') as f:
            content = f.read(100)  # Read just the first 100 chars
            
            # If it starts with a # character, it's a placeholder text file
            if content.strip().startswith('#'):
                print(f"❌ Error: {model_path} is a placeholder text file, not a real model.")
                print("Content preview:")
                print(content)
                return False
    except UnicodeDecodeError:
        # If we can't read it as text, it's likely a binary file (good!)
        print(f"✅ {model_path} appears to be a binary file (likely a real model).")
        return True
        
    # Final check - try loading with YOLO
    try:
        from ultralytics import YOLO
        model = YOLO(model_path)
        print(f"✅ Successfully loaded {model_path} as a YOLO model.")
        return True
    except Exception as e:
        print(f"❌ Error loading model with YOLO: {str(e)}")
        return False

# Check the sample model file
sample_model_path = "/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights/last.pt"
if os.path.exists(sample_model_path):
    check_model_file(sample_model_path)
else:
    print(f"Model file not found: {sample_model_path}")

2. If the model file is invalid, replace it with a real one:

In [ ]:
# Replace an invalid model file with a real one
def replace_invalid_model(model_path):
    print(f"Replacing invalid model at {model_path} with a real YOLOv8n model...")
    
    # Make sure directory exists
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    
    # Download a real YOLOv8n model
    from ultralytics import YOLO
    model = YOLO("yolov8n.pt")
    
    # Remove old file if it exists
    if os.path.exists(model_path):
        os.remove(model_path)
        
    # Save the real model
    model.save(model_path)
    print(f"✅ Successfully replaced with real model at: {model_path}")
    return True

# Uncomment and run if needed
# replace_invalid_model("/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights/last.pt")